# Teil 2: K-Means Clustering

## Zwei Arten zu lernen

Stell dir zwei Situationen vor:

**Situation A - Mit Lehrer (überwachtes Lernen):**
> Ein Lehrer zeigt dir Blumen und sagt: "Das ist eine Setosa, das ist eine Versicolor..."

**Situation B - Ohne Lehrer (unüberwachtes Lernen):**
> Du bekommst 150 Blumen und sollst sie selbst in Gruppen sortieren - ohne zu wissen, wie die Gruppen heissen!

In diesem Notebook probieren wir **Situation B**: Kann ein Computer Gruppen finden, ohne die Antworten zu kennen?

---

## Module und Daten laden

In [ ]:
import sys
import warnings
warnings.filterwarnings("ignore", message=".*as_object_map.*")

if "pyodide" in sys.modules:
    import piplite
    await piplite.install(["numpy", "pandas", "matplotlib", "scikit-learn"])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

df = pd.read_csv("data/iris.csv")
print(f"Daten geladen: {len(df)} Blumen")

## Kannst du die Gruppen selbst erkennen?

Hier siehst du alle 150 Blumen - aber **ohne Farben**! Du weisst nicht, welche Art jede Blume ist.

**Deine Aufgabe:** Schau dir das Diagramm an. Erkennst du Gruppen?

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df["petal_length"], df["petal_width"], c="gray", s=60, alpha=0.7)
plt.xlabel("Blütenblatt-Länge (cm)")
plt.ylabel("Blütenblatt-Breite (cm)")
plt.title("150 Blumen - Welche Gruppen siehst du?")
plt.show()

### Aufgabe 1: Gruppen finden

1. **Wie viele Gruppen** erkennst du im Diagramm?

2. **Beschreibe die Gruppen:** Wo liegen sie? (z.B. "Eine Gruppe unten links...")

3. **Überlappungen:** Sind alle Gruppen klar getrennt?

### Deine Antworten zu Aufgabe 1

**1. Anzahl Gruppen:**

*Doppelklicke hier und schreibe deine Antwort...*

**2. Beschreibung der Gruppen:**

*Doppelklicke hier und schreibe deine Antwort...*

**3. Überlappungen:**

*Doppelklicke hier und schreibe deine Antwort...*

---

## Was ist K-Means?

**K-Means** ist ein Algorithmus, der automatisch Gruppen ("Cluster") in Daten findet.

- **K** = Die Anzahl Gruppen, die wir suchen (wir sagen: "Finde 3 Gruppen")
- **Means** = Mittelwerte (jede Gruppe hat einen Mittelpunkt)

### Die Magneten-Analogie

Stell dir vor:
1. Du hast einen Haufen **Büroklammern** auf dem Tisch (= unsere Datenpunkte)
2. Du wirfst **3 Magnete** zufällig auf den Tisch (= die Startpunkte)
3. Jede Büroklammer wird vom **nächsten Magneten** angezogen
4. Du verschiebst jeden Magneten in die **Mitte seiner Büroklammern**
5. Wiederhole Schritt 3-4, bis sich nichts mehr bewegt

Am Ende hat jeder Magnet eine Gruppe von Büroklammern um sich gesammelt!

### Der Algorithmus

```
1. Setze K zufällige Mittelpunkte
2. Ordne jeden Punkt dem nächsten Mittelpunkt zu
3. Berechne neue Mittelpunkte (Durchschnitt jeder Gruppe)
4. Wiederhole 2-3 bis sich nichts mehr ändert
```

## K-Means Schritt für Schritt

Beobachte, wie K-Means lernt:
- Die **schwarzen X** sind die Mittelpunkte ("Magnete")
- Die **Farben** zeigen, zu welchem Mittelpunkt jede Blume gehört

In [ ]:
X_vis = df[["petal_length", "petal_width"]].values
colors = ["purple", "orange", "cyan"]
titles = ["Start: Zufällige Mittelpunkte", "Nach 1 Runde", "Nach 2 Runden", "Endergebnis"]

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
np.random.seed(42)

for idx, (ax, max_iter) in enumerate(zip(axes.flatten(), [1, 2, 3, 10])):
    km = KMeans(n_clusters=3, max_iter=max_iter, n_init=1, random_state=42, init='random')
    km.fit(X_vis)
    
    for c in range(3):
        ax.scatter(X_vis[km.labels_ == c, 0], X_vis[km.labels_ == c, 1], c=colors[c], alpha=0.6, s=50)
    ax.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1], c="black", marker="X", s=200)
    ax.set(title=titles[idx], xlabel="Blütenblatt-Länge (cm)", ylabel="Blütenblatt-Breite (cm)")

plt.suptitle("K-Means lernt Schritt für Schritt", fontweight="bold")
plt.tight_layout()
plt.show()

### Was ist passiert?

1. **Start:** Die Mittelpunkte (X) starten an zufälligen Positionen
2. **Nach 1 Runde:** Die Mittelpunkte wandern Richtung ihrer Punkte
3. **Nach 2 Runden:** Die Gruppen werden klarer
4. **Endergebnis:** Die Mittelpunkte haben ihre finale Position gefunden

K-Means hat **selbstständig** drei Gruppen gefunden - ohne zu wissen, dass es Blumenarten sind!

## K-Means auf alle Daten anwenden

Jetzt wenden wir K-Means auf alle 4 Messungen an (nicht nur Blütenblatt):

In [ ]:
X_cluster = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(X_cluster)
cluster_labels = kmeans.labels_

print("K-Means hat 3 Gruppen gefunden:")
for i in range(3):
    print(f"   Cluster {i}: {(cluster_labels == i).sum()} Blumen")

## Vergleich: K-Means vs. echte Blumenarten

Jetzt kommt der spannende Teil: Hat K-Means die **echten Blumenarten** gefunden?

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
cluster_colors = ["purple", "orange", "cyan"]
real_colors = ["red", "green", "blue"]

# K-Means Cluster
for c in range(3):
    mask = cluster_labels == c
    ax1.scatter(df.loc[mask, "petal_length"], df.loc[mask, "petal_width"], c=cluster_colors[c], label=f"Cluster {c}", s=60)
ax1.scatter(kmeans.cluster_centers_[:, 2], kmeans.cluster_centers_[:, 3], c="black", marker="X", s=150)
ax1.set(xlabel="Blütenblatt-Länge (cm)", ylabel="Blütenblatt-Breite (cm)", title="K-Means (ohne Labels!)")
ax1.legend()

# Echte Blumenarten
for i, species in enumerate(df["species"].unique()):
    data = df[df["species"] == species]
    ax2.scatter(data["petal_length"], data["petal_width"], c=real_colors[i], label=species, s=60)
ax2.set(xlabel="Blütenblatt-Länge (cm)", ylabel="Blütenblatt-Breite (cm)", title="Echte Blumenarten")
ax2.legend()

plt.tight_layout()
plt.show()

### Wie gut hat es funktioniert?

In [ ]:
print("Zuordnung: Welche echten Blumenarten sind in welchem Cluster?\n")

for cluster in range(3):
    print(f"Cluster {cluster}:", end="  ")
    for species in df["species"].unique():
        count = ((cluster_labels == cluster) & (df["species"] == species)).sum()
        if count > 0:
            print(f"{species}: {count}", end="  ")
    print()

### Aufgabe 2: K-Means verstehen

1. **Vergleiche die Diagramme:** Sehen die K-Means Cluster ähnlich aus wie die echten Blumenarten?

2. **Welche Blumenart** wurde am besten erkannt (fast alle in einem Cluster)?

3. **Wo macht K-Means Fehler?** Welche Arten werden vermischt?

4. **Gedankenexperiment:** Was würde passieren, wenn wir K-Means sagen "finde 2 Gruppen" oder "finde 5 Gruppen"?

### Deine Antworten zu Aufgabe 2

**1. Vergleich der Diagramme:**

*Doppelklicke hier und schreibe deine Antwort...*

**2. Am besten erkannte Blumenart:**

*Doppelklicke hier und schreibe deine Antwort...*

**3. Fehler von K-Means:**

*Doppelklicke hier und schreibe deine Antwort...*

**4. Gedankenexperiment (2 oder 5 Gruppen):**

*Doppelklicke hier und schreibe deine Antwort...*

---

## Zusammenfassung

### Was wir gelernt haben

| Begriff | Bedeutung |
|---------|----------|
| **Clustering** | Daten automatisch in Gruppen einteilen |
| **K-Means** | Ein Algorithmus, der K Gruppen findet |
| **Unüberwachtes Lernen** | Lernen ohne die "richtigen Antworten" zu kennen |
| **Überwachtes Lernen** | Lernen mit bekannten Antworten |

### K-Means: Stärken und Schwächen

**Stärken:**
- Findet Gruppen ohne Labels
- Einfach zu verstehen
- Schnell

**Schwächen:**
- Man muss K (Anzahl Gruppen) vorher wissen
- Findet nur "runde" Cluster
- Ergebnis hängt von Startpunkten ab

### Die grosse Frage

K-Means kann Gruppen finden, **ohne die Antworten zu kennen**. Das ist beeindruckend!

Aber: **Wir HABEN ja die richtigen Antworten!** Wir wissen, welche Blume zu welcher Art gehört.

→ Warum sollten wir dieses Wissen nicht nutzen?

**Weiter geht's:** Im nächsten Notebook trainieren wir ein **neuronales Netz**, das aus den richtigen Antworten lernt!